In [13]:
import json
import requests

In [2]:
url = "http://127.0.0.1:4050/ingest-files"

payload = json.dumps({
    "bucket_name": "rag-accelerator-2.0",
    "connection_name": "milvus_connect",
    "directory": "watsonx_docs",
    "index_name": "sample"
})

response = requests.request("POST", url, data=payload)

print(response.text)

{"status":"success","message":"Data Ingestion successful for 10 document chunks into vector database!"}


In [57]:
url = "http://127.0.0.1:4050/query"

q_payload = json.dumps({
    "connection_name": "milvus_connect",
    "index_name": "sample",
    "query": "How to perform decision optimization?"
})

q_response = requests.request("POST", url, data=q_payload)

print(q_response.text)

{"data":{"answer":[{"id":null,"metadata":{"title":"Decision Optimization migration","source":"watsonx_docs/migration.html","document_url":"https://www.ibm.com/docs/SSYOK8/DO/WML_Deployment/migration.html","page_number":"","chunk_seq":1,"id":"8d0ccd0a187f50b2f19fde4daff510e4f85a8ecb21aef5315ef97f1748dd652e"},"page_content":"Document Title: Decision Optimization migration\n Document Content: Decision Optimization migration\n\n\n\nMigrating\nYou can find different migration guides in this section.\n\n\n\nMigrating from COS or DB2 to a connection asset\nYou can use a connection asset to access COS or DB2 data assets. \nMigrating from Watson Machine Learning API V4 Beta\nAs the Watson Machine Learning V4 API now released, if you previously used the Watson Machine Learning  V4 Beta API for Decision Optimization, read here about some changes and improvements.\nMigrating Python code for Decision Optimization with Machine Learning-v2 instances\nUse the latest version of the Watson Machine Learn

In [58]:
# QnA Queries
url = "http://127.0.0.1:4050/ai/qna/query"

payload = json.dumps({
    "question" : "how to perform decision optimization?"   
})

response = requests.request("POST", url, data=payload)

print(response.text)

{"answer":{"response":"To perform decision optimization, you need to follow a structured approach. First, define the decision problem and identify the key objectives and constraints. Then, gather relevant data and create a mathematical model that represents the decision-making process. Next, use optimization algorithms, such as linear or nonlinear programming, to find the optimal solution. Consider using tools like decision trees, sensitivity analysis, or simulation models to evaluate different scenarios and uncertainty. Finally, validate the results and refine the model as needed to ensure the optimal decision is made. Additionally, consider using machine learning or artificial intelligence techniques to improve the decision-making process.","Hallucination Detection":"NA"},"documents":{},"expert_answer":"No expert profile found for your question in the available documents.","log_id":""}


In [59]:
response_json = json.loads(response.text)
payload_dict = json.loads(payload)

question = payload_dict['question']
answer_text = response_json.get('answer', {}).get('response', '')
documents = response_json.get('documents', {})

contexts_json = json.loads(q_response.text)
top_context = contexts_json.get('data', {}).get('answer')[0]['page_content']
top_context

'Document Title: Decision Optimization migration\n Document Content: Decision Optimization migration\n\n\n\nMigrating\nYou can find different migration guides in this section.\n\n\n\nMigrating from COS or DB2 to a connection asset\nYou can use a connection asset to access COS or DB2 data assets. \nMigrating from Watson Machine Learning API V4 Beta\nAs the Watson Machine Learning V4 API now released, if you previously used the Watson Machine Learning  V4 Beta API for Decision Optimization, read here about some changes and improvements.\nMigrating Python code for Decision Optimization with Machine Learning-v2 instances\nUse the latest version of the Watson Machine Learning Python client for Decision Optimization.\nMigrating from Decision Optimization on Cloud (DOcplexcloud)\nDescribes the difference in workflow when using Decision Optimization for Watson Machine Learning compared with Decision Optimization on Cloud (DOcplexcloud)\n\n\nParent topic: Decision Optimization'

In [ ]:
from dotenv import load_dotenv
import os

from ibm_watsonx_gov.config import Credentials
from ibm_watsonx_gov.entities.enums import Region

from ibm_watsonx_gov.config import GenAIConfiguration
from ibm_watsonx_gov.metrics import ContextRelevanceMetric, FaithfulnessMetric, AnswerRelevanceMetric
from ibm_watsonx_gov.entities.enums import MetricGroup

load_dotenv()

question_field = "question"
context_field = "contexts"

config = GenAIConfiguration(
    input_fields=[question_field],
    context_fields=[context_field],
    output_fields=["answer"],
    reference_fields=["ground_truth"]
)

In [61]:
from ibm_watsonx_gov.clients.api_client import APIClient
from ibm_watsonx_gov.evaluators import MetricsEvaluator

credentials = Credentials(
    api_key=os.getenv("IBM_API_KEY"),
    service_instance_id=os.getenv("WXG_SERVICE_INSTANCE_ID"),
)
evaluator = MetricsEvaluator(api_client = APIClient(credentials=credentials), configuration=config)

governance_data = {
    'question': [question],
    'contexts': [[top_context]], # List of lists
    'answer': [answer_text],
}

In [63]:
metrics = [
    ContextRelevanceMetric(),
    FaithfulnessMetric(),
    AnswerRelevanceMetric(),
]

metric_groups = [
    MetricGroup.RETRIEVAL_QUALITY,
    #MetricGroup.ANSWER_QUALITY,
    #MetricGroup.CONTENT_SAFETY
]

In [ ]:
evaluation_result = evaluator.evaluate(
    data=governance_data,
    metrics=metrics,
    #metric_groups=metric_groups
)

evaluation_result.to_df()

context_relevance.token_precision  faithfulness.token_k_precision  \
0                                0.6                          0.2111   

   answer_relevance.token_recall  
0                            0.8